In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
# utilities
import re
import numpy as np
import pandas as pd
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
from collections import Counter

# Importing Dataset

DATASET_COLUMNS=['target','ids','date','flag','user','text']
DATASET_ENCODING = "ISO-8859-1"
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)
df.sample(5)

# Text Preprocessing

data=df[['text','target']]

data['target'] = data['target'].replace(4,1)

data['target'].unique()

data_pos = data[data['target'] == 1]
data_neg = data[data['target'] == 0]

data_pos = data_pos.iloc[:int(20000)]
data_neg = data_neg.iloc[:int(20000)]

dataset = pd.concat([data_pos, data_neg])

#dataset = data

/tmp/ipykernel_19/2950439869.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['target'].replace(4,1)


In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import pandas as pd

# Assuming 'dataset' is your DataFrame
# Make sure the 'text' column exists in your dataset

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Extract the 'text' column from the dataset
sentences = dataset['text'].tolist()

# Tokenize and process each sentence
tokenized_texts = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in sentences]

# Pad sequences to have the same length
max_len = max(len(tokenized_text) for tokenized_text in tokenized_texts)
padded_tokenized_texts = [tokenized_text + [0] * (max_len - len(tokenized_text)) for tokenized_text in tokenized_texts]


input_ids = torch.tensor(padded_tokenized_texts)

# Install necessary libraries
# pip install transformers
# pip install torch
# pip install pandas
# pip install tqdm

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import pandas as pd

# Assuming 'dataset' is your DataFrame
# Make sure the 'text' column exists in your dataset

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Extract the 'text' column from the dataset
sentences = dataset['text'].tolist()

# Tokenize and process each sentence
tokenized_texts = [tokenizer.encode(sentence, add_special_tokens=True) for sentence in sentences]

# Pad sequences to have the same length
max_len = max(len(tokenized_text) for tokenized_text in tokenized_texts)
padded_tokenized_texts = [tokenized_text + [0] * (max_len - len(tokenized_text)) for tokenized_text in tokenized_texts]

# Convert to PyTorch tensors
input_ids = torch.tensor(padded_tokenized_texts)


# Set batch size
batch_size = 8

# Perform inference with the BERT model in batches
predicted_sentiments = []

with torch.no_grad(), tqdm(total=len(input_ids)) as pbar:
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i + batch_size]
        batch_outputs = model(batch_input_ids)
        batch_sentiment_scores = batch_outputs.logits
        batch_sentiment_probabilities = torch.nn.functional.softmax(batch_sentiment_scores, dim=-1)
        batch_predicted_sentiments = torch.argmax(batch_sentiment_probabilities, dim=-1).tolist()
        predicted_sentiments.extend(batch_predicted_sentiments)
        pbar.update(len(batch_input_ids))

# Add predicted sentiments to the original dataset
dataset['predicted_sentiment'] = predicted_sentiments

# Print the updated dataset
print(dataset[['text', 'predicted_sentiment']])

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 40000/40000 [1:57:42<00:00,  5.66it/s]

                                                     text  predicted_sentiment
800000       I LOVE @Health4UandPets u guys r the best!!                     1
800001  im meeting up with one of my besties tonight! ...                    1
800002  @DaRealSunisaKim Thanks for the Twitter add, S...                    1
800003  Being sick can be really cheap when it hurts t...                    1
800004    @LovesBrooklyn2 he has that effect on everyone                     1
...                                                   ...                  ...
19995   Not much time off this weekend, work trip to M...                    1
19996                           One more day of holidays                     1
19997   feeling so down right now .. i hate you DAMN H...                    1
19998   geez,i hv to READ the whole book of personalit...                    1
19999   I threw my sign at donnie and he bent over to ...                    1

[40000 rows x 2 columns]


In [4]:
# Assuming 'dataset' is your DataFrame with the updated predictions

# Define the output CSV file path
output_csv_path = 'predicted_sentiments_dataset.csv'

# Export the dataset to CSV
dataset.to_csv(output_csv_path, index=False)

# Print a message indicating the export is complete
print(f'Dataset with predicted sentiments exported to {output_csv_path}')


Dataset with predicted sentiments exported to predicted_sentiments_dataset.csv


In [1]:
from sklearn.metrics import f1_score, classification_report

# Compare predicted and original values
original_sentiments = dataset['target'].tolist() 

# Calculate F1 score
f1 = f1_score(original_sentiments, predicted_sentiments, average='weighted')

print(f'F1 Score: {f1 * 100:.2f}%')

F1 Score: 93.67%
